<a href="https://colab.research.google.com/github/crystalloide/Spark/blob/master/Notebook_exemple_de_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

# !pip install IPython
# !apt install openjdk-8-jdk
# !apt autoremove 
# !java -version
# !help

**Spark SQL** 

Spark SQL est un composant au-dessus de Spark Core qui facilite le traitement des données structurées et semi-structurées et l'intégration de plusieurs comme source de formats de données (Hive, Parquet, JSON). 

Il permet de transformer des RDD en utilisant SQL (Structured Query Language). 

Pour démarrer Spark SQL dans votre bloc-notes, vous devez créer un contexte SQL. 

Pour cet exercice, nous allons importer un fichier JSON contenant des informations de la «Coupe du monde de football». 

Remarque : notre notebook jupyter doit utiliser un noyau en python 3 
=> Penser à changer le noyau s'il est défini sur python 2.


In [2]:
%ls
%pwd
# % rm World*.*
# % rm *.json
# %ls


all-world-cup-players.json  sample_data/  spark-warehouse/


'/content'

In [0]:
# Au cas où vous souhaiteriez traiter un fichier stocké dans google drive 
# from google.colab import drive
# drive.mount('/Fichiers')

In [0]:
# Ici on va récupérer un fichier précis sur le poste local et l'uploader ensuite dans /content sur google colaboratory 
# Décommentez ci-dessous pour refaire l'upload :
# from google.colab import files
# files.upload()


In [4]:
%ls
# drive.flush_and_unmount()

all-world-cup-players.json  sample_data/  spark-warehouse/


In [0]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local', 'Spark SQL') 
sqlc = SQLContext(sc)


Nous pouvons lire le fichier JSON que nous avons et créer un DataFrame (Spark SQL a un lecteur json disponible):

In [6]:
# Si on voulait lire un fichier csv : 
# players = sqlc.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('WorldCupPlayers.csv')

# Mais ici, on veut lire un fichier json et - en plus- en multi-ligne :
players = sqlc.read.option("multiline", "true").json("all-world-cup-players.json")

# Affiche le schema sous forme d'arborescence
players.printSchema()

# Nous pouvons alors interroger notre vue :

# Selectionne uniquement la colonne "FullName"
players.select('FullName').show(20)

# Et pour obtenir les noms des équipes :
players.select('Team').show(2)

root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)

+--------------------+
|            FullName|
+--------------------+
|        Ãngel Bossio|
|        Juan Botasso|
|      Roberto Cherro|
|   Alberto Chividini|
|                    |
|                    |
|       Juan Evaristo|
|      Mario Evaristo|
|     Manuel Ferreira|
|          Luis Monti|
|                    |
|   Rodolfo Orlandini|
|Fernando Paternoster|
|   Natalio Perinetti|
|     Carlos Peucelle|
|     Edmundo Piaggio|
|  Alejandro Scopelli|
|      Carlos Spadaro|
|                    |
|                    |
+--------------------+
only showing top 20 rows

+---------+
|     Tea

Ensuite, nous pouvons créer une vue de notre DataFrame. 

La durée de vie de cette table temporaire est liée à la SparkSession qui a été utilisée pour créer ce DataFrame.

In [0]:
players.registerTempTable("players_sql")

Et nous avons toutes les possibilités du SQL pour créer une requête :

In [8]:
query = """SELECT * FROM players_sql"""
teamcomplete = sqlc.sql( query )
teamcomplete.show()

+--------------------+-----------+-----------+-----------+--------------------+---------+------+--------+---------+----+
|                Club|ClubCountry|Competition|DateOfBirth|            FullName|IsCaptain|Number|Position|     Team|Year|
+--------------------+-----------+-----------+-----------+--------------------+---------+------+--------+---------+----+
|Club AtlÃ©tico Ta...|  Argentina|  World Cup|   1905-5-5|        Ãngel Bossio|    false|      |      GK|Argentina|1930|
|Quilmes AtlÃ©tico...|  Argentina|  World Cup| 1908-10-23|        Juan Botasso|    false|      |      GK|Argentina|1930|
|          Boca Junio|  Argentina|  World Cup|  1907-2-23|      Roberto Cherro|    false|      |      FW|Argentina|1930|
|Central Norte TucumÃ|  Argentina|  World Cup|  1907-2-23|   Alberto Chividini|    false|      |      DF|Argentina|1930|
|Club Atletico Est...|  Argentina|  World Cup|  1909-3-19|                    |    false|    10|      FW|Argentina|1930|
|Racing Club de Av...|  Argentin

In [9]:
# Selection des noms des équipes depuis 2014 seulement  

query2014 = """SELECT DISTINCT Team, YEAR FROM players_sql WHERE Year == 2014"""
team2014 = sqlc.sql( query2014 )
team2014.show(50)

# Autre écriture qui -elle- ne fonctionnera pas : 
# team2014 = sqlc.sql("SELECT DISTINCT Team, YEAR FROM players_sql WHERE Year == 2014")

# Les résultats des requêtes SQL sont des Dataframes.


+--------------------+----+
|                Team|YEAR|
+--------------------+----+
|            Colombia|2014|
|             Croatia|2014|
|         Netherlands|2014|
|                Iran|2014|
|             Uruguay|2014|
|              Greece|2014|
|Bosnia and Herzeg...|2014|
|          Costa Rica|2014|
|             Ecuador|2014|
|           Argentina|2014|
|           Australia|2014|
|              Mexico|2014|
|              France|2014|
|             Algeria|2014|
|               Ghana|2014|
|       United States|2014|
|         Switzerland|2014|
|            Honduras|2014|
|              Russia|2014|
|               Chile|2014|
|         South Korea|2014|
|               Italy|2014|
|             Nigeria|2014|
|         Ivory Coast|2014|
|               Japan|2014|
|              Brazil|2014|
|             Belgium|2014|
|               Spain|2014|
|             Germany|2014|
|            Portugal|2014|
|            Cameroon|2014|
|             England|2014|
+-------------------